In [1]:
AGENT_NAME = 'default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500'
ADVERSARY_PATH = 'Models\Adversary\default_adversary_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_default_rwd_480'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
MAXITER = 500
INIT = 20
EPS = 0.3 #comparable to the mean BB Linf distortion
NAME = f'Optimal ACG eps {EPS} max iter {MAXITER}'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.attacks.evasion import AutoConjugateGradient as ACG

import pandas as pd
import numpy as np
import os

import KBMproject.utilities as utils

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

Define RL agent

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
print('Model loaded from storage')

Model loaded from storage


In [5]:
adv_path = os.path.join(os.getcwd(), '..',ADVERSARY_PATH) #get adversary from parent directory
adv_path = os.path.normpath(adv_path) #resolve '..'
adversary = PPO.load(path=adv_path)
print('Model loaded from storage')

Model loaded from storage


In [6]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [7]:
observation_masks = pd.Series(1, index=env.observation_names[0]).to_frame(name='all')
observation_masks['mask_time'] = observation_masks['all']
observation_masks['mask_time'][observation_masks.index.str.contains('month|day|hour')] = 0

In [8]:
%%time
kwargs = dict(
    loss_type='difference_logits_ratio', 
    batch_size=1,
    nb_random_init=INIT, #5, lower values speed crafting
    max_iter=MAXITER//INIT, #iterations per restart
    norm='inf', #->l2 ->l1 most restrictive
    targeted=True,
    eps=EPS, 
    eps_step=2*EPS, #larger starting step for more diversification
    verbose=False,
)
Adv_targered_kpis, obs, adv_obs, actions, targets, asr = utils.eval_targeted_attack(agent,
                                                                                    adversary,
                                                                                    env,
                                                                                    ART_atk=utils.define_attack(agent, ACG, **kwargs ),
                                                                                    time_steps=10,
                                                                                    mask=observation_masks['mask_time'].to_numpy())
display(Adv_targered_kpis)
print(f'The ASR is {asr}')


100%|██████████| 10/10 [00:21<00:00,  2.15s/it, ASR: =0]


ValueError: too many values to unpack (expected 4)

In [ ]:
# df_targeted_obs = pd.DataFrame(obs)
# df_targeted_obs.to_csv(f'{NAME} observations.csv')
# df_targeted_obs = pd.DataFrame(adv_obs)
# df_targeted_obs.to_csv(f'{NAME} adversarial observations.csv')

In [ ]:
df_kpis = pd.read_csv('KPIs.csv', index_col='cost_function')
df_kpis

In [14]:
df_kpis[f'{NAME}'] = Adv_targered_kpis.values

In [ ]:
df_kpis[['baseline','untargeted_acg',f'{NAME}']].plot.barh(xlim=0.7).legend(bbox_to_anchor=(1.0, 1.05))

In [23]:
#df_kpis.to_csv('KPIs.csv')